In [1]:
!pip install selenium

You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install bs4

Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
    Running setup.py install for bs4: started
    Running setup.py install for bs4: finished with status 'done'


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
# Import libraries and packages for the project 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv
print('- Finish importing packages')

- Finish importing packages


In [7]:
# Task 1.1: Open Chrome and Access Linkedin login site
path='E:/Download/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(path)
sleep(2)
url = 'https://www.linkedin.com/login'
driver.get(url)
print('- Finish initializing a driver')
sleep(2)

C:\Users\Admin\AppData\Local\Temp/ipykernel_8960/928450213.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


- Finish initializing a driver


In [9]:
credential = open('credentials.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('- Finish importing the login credentials')
sleep(2)
# Task 1.2: Key in login credentials
email_field = driver.find_element_by_id('username')
email_field.send_keys(username)
print('- Finish keying in email')
sleep(3)

password_field = driver.find_element_by_name('session_password')
password_field.send_keys(password)
print('- Finish keying in password')
sleep(2)

# Task 1.2: Click the Login button
signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
signin_field.click()
sleep(3)

print('- Finish Task 1: Login to Linkedin')

- Finish importing the login credentials


C:\Users\Admin\AppData\Local\Temp/ipykernel_8960/2902101160.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email_field = driver.find_element_by_id('username')


- Finish keying in email


C:\Users\Admin\AppData\Local\Temp/ipykernel_8960/2902101160.py:13: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  password_field = driver.find_element_by_name('session_password')


- Finish keying in password


C:\Users\Admin\AppData\Local\Temp/ipykernel_8960/2902101160.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')


In [11]:

# Task 2: Search for the profile we want to crawl
# Task 2.1: Locate the search bar element
search_field = driver.find_element_by_xpath('//*[@id="global-nav-typeahead"]/input')
# //*[@id="global-nav-typeahead"]/input

# Task 2.2: Input the search query to the search bar
search_query = input('What type of key want to search ')
search_field.send_keys(search_query)

# Task 2.3: Search
search_field.send_keys(Keys.RETURN)

print('- Finish Task 2: Search for profiles')

C:\Users\Admin\AppData\Local\Temp/ipykernel_8960/20798977.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_field = driver.find_element_by_xpath('//*[@id="global-nav-typeahead"]/input')


What type of key want to search Data engineer
- Finish Task 2: Search for profiles


In [13]:
# Task 3: Scrape the URLs of the profiles
# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link') #('a', class_ = 'search-result__result-link ember-view')
    all_profile_URL = []
    for profile in profiles:
        # profile_ID = profile.get('href')
        # profile_URL = "https://www.linkedin.com" + profile_ID
        profile_URL = profile.get('href')
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

In [14]:
# Task 3.2: Navigate through many page, and extract the profile URLs of each page
input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    next_button = driver.find_element_by_class_name("artdeco-pagination__button--next")
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)

print('- Finish Task 3: Scrape the URLs')

How many pages you want to scrape: 1


C:\Users\Admin\AppData\Local\Temp/ipykernel_8960/1475792147.py:9: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  next_button = driver.find_element_by_class_name("artdeco-pagination__button--next")


- Finish Task 3: Scrape the URLs


In [22]:
import random # add sleeptime to scale system, and pass blocking by pass
import pandas as pd
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL) # send output to comment 
        sleep(3) # random range
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        try:
            name = info_div.find('li', class_='inline t-24 t-black t-normal break-words').get_text().strip() #Remove unnecessary characters 
            print('--- Profile name is: ', name)
            location = info_div.find('li', class_='t-16 t-black t-normal inline-block').get_text().strip() #Remove unnecessary characters 
            print('--- Profile location is: ', location)
            title = info_div.find('h2', class_='mt1 t-18 t-black t-normal break-words').get_text().strip()
            print('--- Profile title is: ', title)
            writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
            print('\n')
        except:
            pass

print('Mission Completed!')


WebDriverException: Message: chrome not reachable
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FA9943+2595139]
	Ordinal0 [0x00F3C9F1+2148849]
	Ordinal0 [0x00E343F0+1065968]
	Ordinal0 [0x00E287C2+1017794]
	Ordinal0 [0x00E28FF8+1019896]
	Ordinal0 [0x00E2A892+1026194]
	Ordinal0 [0x00E24219+999961]
	Ordinal0 [0x00E35860+1071200]
	Ordinal0 [0x00E8B2D2+1422034]
	Ordinal0 [0x00E7B806+1357830]
	Ordinal0 [0x00E56086+1204358]
	Ordinal0 [0x00E56F96+1208214]
	GetHandleVerifier [0x0114B232+1658114]
	GetHandleVerifier [0x0120312C+2411516]
	GetHandleVerifier [0x0103F261+560433]
	GetHandleVerifier [0x0103E366+556598]
	Ordinal0 [0x00F4286B+2173035]
	Ordinal0 [0x00F475F8+2192888]
	Ordinal0 [0x00F476E5+2193125]
	Ordinal0 [0x00F511FC+2232828]
	BaseThreadInitThunk [0x760C6739+25]
	RtlGetFullPathName_UEx [0x773D8E7F+1215]
	RtlGetFullPathName_UEx [0x773D8E4D+1165]


In [ ]:
with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)
        sleep(3)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        try:
            name = info_div.find('li', class_='inline t-24 t-black t-normal break-words').get_text().strip() #Remove unnecessary characters 
            print('--- Profile name is: ', name)
            location = info_div.find('li', class_='t-16 t-black t-normal inline-block').get_text().strip() #Remove unnecessary characters 
            print('--- Profile location is: ', location)
            title = info_div.find('h2', class_='mt1 t-18 t-black t-normal break-words').get_text().strip()
            print('--- Profile title is: ', title)
            writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
            print('\n')
        except:
            pass

print('Mission Completed!')